### Importe

In [3]:
import cv2
import numpy as np
import imutils
import math

from tensorflow.keras.layers import Dense, Conv2D, Flatten, MaxPool2D, Dropout
from tensorflow.keras.models import Sequential

### Index based Segmentation

By using the Excessive green index, plant and backgrounds pixels are segemented. In addition, Otsu thresholding is used to determine the threshold for plants and background.



In [6]:
# Read single image for sample size
image_mat = cv2.imread(f'../image.JPG')

# Compute excessive green index (ExG) for every plant pixel
exg_factors = np.full((image_mat.shape[0], image_mat.shape[1], image_mat.shape[2]), [1, -2, 1])
segmentation = np.sum(image_mat*exg_factors, 2)
# Normalize pixel values with a max value of 255
segmentation = cv2.normalize(src=segmentation, dst=None, alpha=0, beta=255, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_8U)
# Use Otsu to identify threshold between plant and background pixels in grayscale mask
ret, background_mask = cv2.threshold(segmentation, 0, 255, cv2.THRESH_OTSU)
ret, plant_mask = cv2.threshold(background_mask, 0, 255, cv2.THRESH_BINARY_INV)

plant_mask = np.array(plant_mask, np.uint8)
background_mask = np.array(background_mask, np.uint8)


### Hough transformation to identify crop rows

In order to automatically extract training data from the drone images, crop rows are computed. Every plant that crosses the crop rows is identified as crop. Therefore, a hough transformation is used. As preparation, contours of the plants are extracted for efficient computation. Additionally, outlier plants are extracted by contour size because we assume all plants have roughly the same size. This reduces computional complexity even further and yields a clean picture.

In [11]:
def draw_lines(img, houghLines, color=[255, 255, 255], thickness=1):
    """Draw lines on the image which can be used to determine intersections between lines and plant contours.
    """
    for line in houghLines:
        for rho,theta in line:
            a = np.cos(theta)
            b = np.sin(theta)
            x0 = a*rho
            y0 = b*rho
            x1 = int(x0 + 10000*(-b))
            y1 = int(y0 + 10000*(a))
            x2 = int(x0 - 10000*(-b))
            y2 = int(y0 - 10000*(a))
            cv2.line(img,(x1,y1),(x2,y2),color,thickness)
            

def draw_contours(contours, shape, color = (255, 255, 255), thickness = 1):
    """Utitlity function to draw contours on black image.
    """
    black_mat = np.zeros((shape[0],shape[1]), np.uint8)
    for c in contours:
        cv2.drawContours(black_mat, [c], -1, color, thickness)
    return black_mat

def calculate_main_theta(lines):
    """Calculate main angle for crop rows based on hough lines.
    """
    thetas = lines[:,0,1]
    hist = np.histogram(thetas, 180, (0, np.pi))
    index = hist[0].argmax()
    return index * np.pi / 180

In [34]:
# Parameters of Hough Lines transformation
rho_resolution = 1
theta_resolution = np.pi / 180
threshold = 120
tolerance = np.pi / 180 * 1.5
exclude_contours_factor = 0.65

In [35]:
# Extract plant contours from image
contours = cv2.findContours(plant_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
contours = imutils.grab_contours(contours)
# Identify outliers and remove from contours
contour_areas = [cv2.contourArea(c) for c in contours]
contour_areas = sorted(contour_areas)
outlier_index = math.ceil(len(contour_areas) * exclude_contours_factor)
clean_contour_areas = contour_areas[outlier_index:len(contour_areas)]
clean_contours = [c for c in contours if (cv2.contourArea(c)) > clean_contour_areas[0]]

Identify crop rows based on cleaned contours. Initially, contours are placed onto a black image which is used to identify rows. In order to extract acutal crop rows, the main angle of the hough lines is calculated. Only hough lines are saved which angle is around a specified threshold to the main angle.

In [36]:
# Draw contours of plants
black_image = np.zeros(plant_mask.shape, np.uint8)
for c in clean_contours:
    cv2.drawContours(black_image, [c], -1, (255, 255), 1)
edges = cv2.Canny(black_image,50,150,apertureSize = 7)
non_zero_edges_pixels = cv2.findNonZero(edges)
# Compute hough lines
lines = cv2.HoughLines(edges, rho_resolution , theta_resolution , threshold)
hough_lines_image = np.zeros(plant_mask.shape, np.uint8)
if lines is None:
        print("No crop rows found.")
else:
    main_theta = calculate_main_theta(lines)
    min_theta = main_theta - tolerance
    max_theta = main_theta + tolerance
    good_lines = cv2.HoughLines(edges, rho_resolution , theta_resolution , threshold, min_theta=min_theta, max_theta=max_theta)
    if good_lines is None:
        print("No good crop rows found.")
    else:
        draw_lines(hough_lines_image, good_lines)

Based on the identified crop rows, we extract plants that intersect with these rows. This way, training data can be extracted from all aerial images automatically and we save to label those by hand.

In [ ]:
contour_mask = draw_contours(clean_contours, hough_lines_image.shape)
intersections = cv2.bitwise_and(contour_mask, hough_lines_image)
intersection_points = cv2.findNonZero(intersections)
crops_on_row_contours = []
crops_out_of_row_contours = []
for c in clean_contours:
    contour_on_row = False
    y_max = max(c[:,0,-1])
    y_min = min(c[:,0,-1])
    x_max = max(c[:,0,0])
    x_min = min(c[:,0,0])
    for intersection_point in intersection_points:
        if not contour_on_row:
            if (intersection_point[0][0] >= x_min and intersection_point[0][0] <= x_max 
                and intersection_point[0][1] >= y_min 
                and intersection_point[0][1] <= y_max):
                    dist = cv2.pointPolygonTest(c,
                                                (intersection_point[0][0],
                                                 intersection_point[0][1]),
                                                False)
                    if dist == 0:
                        crops_on_row_contours.append(c)
                        contour_on_row = True
    if not contour_on_row:
        crops_out_of_row_contours.append(c)

### CNN Architecture

In [7]:
def initialize_model():
    """Define CNN architecture."""
    model = Sequential()
    model.add(Conv2D(192, kernel_size=5, activation='relu', padding= 'same', input_shape=(32, 32, 3)))
    model.add(MaxPool2D())
    model.add(Conv2D(256, kernel_size=5, activation='relu', input_shape=(16, 16, 3)))
    model.add(MaxPool2D())
    model.add(Conv2D(256, kernel_size=3, activation='relu', input_shape=(8, 8 , 3)))
    model.add(MaxPool2D())
    model.add(Flatten())
    model.add(Dense(512, activation ='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(512, activation ='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(2, activation= 'softmax'))
    return model